In [1]:
from tensorpack.predict.config import PredictConfig
from tensorpack.tfutils import SmartInit

OpenCV is built with OpenMP support. This usually results in poor performance. For details, see https://github.com/tensorpack/benchmarks/blob/master/ImageNet/benchmark-opencv-resize.py


In [2]:
from VAE.Model import ModelDesc, Trainer  # , RandomZData
from VAE.extra_info_params import get_default_hparams

hps = get_default_hparams()

In [3]:
hps.T

60

In [4]:
hps.D

46

In [5]:
hps.data_file_name

'/home/nam/data/ccxt/preprocessing_data/BTC_USDT_binance_5m.csv_train'

In [6]:
checkpoint_path = './train_log/train_5m/checkpoint'

In [7]:
M = ModelDesc(hps)
pred_config = PredictConfig(
        session_init=SmartInit(checkpoint_path),
        model=M,
        input_names=['x'],
        output_names=['predict/y_pred']
)

In [8]:
from tensorpack.predict import OfflinePredictor
predictor = OfflinePredictor(pred_config)

[0101 20:08:25 @collection.py:146] New collections created in tower : tf.GraphKeys.MODEL_VARIABLES of size 10, tf.GraphKeys.METRIC_VARIABLES of size 2
[0101 20:08:25 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->7)
[0101 20:08:25 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the graph: global_step, optimize/beta1_power, optimize/beta2_power
[0101 20:08:26 @sessinit.py:114] Restoring checkpoint from ./train_log/train_5m/model-13410 ...
INFO:tensorflow:Restoring parameters from ./train_log/train_5m/model-13410


In [9]:
import pandas as pd
import numpy as np

In [10]:
from VAE.load_data import *
def get_data(hps):
    dfX, df_next_deltaClose = load_data_seq(hps)

    segment, next_segment, target_one_hot = segment_seq(dfX, df_next_deltaClose, hps)

    train_segment, test_segment, _, _, train_target_one_hot, test_target_one_hot =\
    train_test_split(segment, next_segment, target_one_hot, hps)

    return train_segment, train_target_one_hot, test_segment, test_target_one_hot

In [11]:
X_train, y_train, X_val, y_val = get_data(hps)

Normalize: Z score


In [12]:
X_train.shape

(184460, 60, 46)

In [13]:
y_train.shape

(184460, 1, 2)

In [14]:
X_val.shape

(25860, 60, 46)

In [15]:
cols = hps.attributes_normalize_mean
test_file = '/home/nam/data/ccxt/preprocessing_data/BTC_USDT_binance_5m.csv_test'
df_X_test = pd.read_csv(test_file)
X_test_seq = df_X_test[cols].values
T, D = hps.T, hps.D

In [16]:
df_X_test.tail()

,Timestamp,Open,High,Low,Close,Volume,N_buy,N_sell,buy_amount_avg,sell_amount_avg,...,_1d_Spread_Open_Close,_1d_Spread_High_Low,_1d_Spread_Buy_Sell,_1d_Spread_Buy_Sell_avg,_1d_MA_Close_6,_1d_MA_Close_12,_1d_MA_Close_288,_1d_MA_Close_2880,1d_target,trend
8635,2019-09-30 23:35:00,8249.17,8277.57,8248.45,8263.75,108.150063,7710.0,6965.0,0.153769,0.117014,...,-2.28,13.06,-44.016928,-0.004544,5.490000,3.616667,0.705694,-0.663861,16.25,1
8636,2019-09-30 23:40:00,8264.56,8289.00,8259.01,8280.00,125.008526,7555.0,6943.0,0.156957,0.119180,...,-0.86,0.87,-12.208191,0.001023,6.225000,5.742500,0.802708,-0.658892,-9.53,0
8637,2019-09-30 23:45:00,8279.75,8282.99,8260.86,8270.47,64.111605,7151.0,6778.0,0.158853,0.119083,...,24.72,-7.86,-29.535548,0.001992,3.715000,4.672500,0.752708,-0.659878,-6.04,0
8638,2019-09-30 23:50:00,8270.71,8270.71,8256.01,8264.43,69.797425,6974.0,6645.0,0.163158,0.120201,...,-3.00,-7.43,10.318984,0.003188,3.221667,3.237500,0.730035,-0.661687,24.91,1
8639,2019-09-30 23:55:00,8264.19,8295.00,8263.56,8289.34,115.803928,6801.0,6508.0,0.163959,0.122547,...,-31.43,16.74,-21.577561,-0.001545,8.351667,5.896667,0.852500,-0.652517,-47.01,0


In [17]:
X_test_seq.shape

(8640, 46)

In [18]:
N_seq = X_test_seq.shape[0]
N_test = N_seq - T + 1
X_test = np.zeros(shape=(N_test, T, D))
for i in range(N_test):
    X_test[i] = X_test_seq[i:i+T]

In [19]:
scaler = np.load(hps.scaler_path)
mu = scaler['mu']
# X_min = scaler['X_min']
# X_max = scaler['X_max']
X_std = scaler['X_std']
X_test = (X_test - mu) / X_std

In [20]:
y_test = df_X_test.trend[T-1:N_test + T -1].values # shift T -1
y_test

array([1, 0, 1, ..., 0, 1, 0])

In [21]:
# predict training set
M = hps.M
N_train = X_train.shape[0]  # 2160
y_train_hat = np.zeros([N_train, 1, hps.C])
y_train_hat[-M:] = predictor(X_train[-M:])[0]

for i in range(N_train // M):
    idx = M * i
    y_train_hat[M * i:M *(i+ 1)] = predictor(X_train[M * i:M *(i+ 1)])[0]
    
y_train_hat = np.argmax(y_train_hat[:,0,:], axis = -1)
y_train_hat

array([0, 1, 1, ..., 1, 1, 1])

In [22]:
# predict validation set
M = hps.M
N_val = X_val.shape[0]  # 2160
y_val_hat = np.zeros([N_val, 1, hps.C])
y_val_hat[-M:] = predictor(X_val[-M:])[0]

for i in range(N_val // M):
    idx = M * i
    y_val_hat[M * i:M *(i+ 1)] = predictor(X_val[M * i:M *(i+ 1)])[0]
    
y_val_hat = np.argmax(y_val_hat[:,0,:], axis = -1)
y_val_hat

array([0, 0, 1, ..., 0, 0, 0])

In [23]:
# predict test
M = hps.M

y_test_hat = np.zeros([N_test, 1, hps.C])
y_test_hat[-M:] = predictor(X_test[-M:])[0]

for i in range(N_test // M):
    idx = M * i
    y_test_hat[M * i:M *(i+ 1)] = predictor(X_test[M * i:M *(i+ 1)])[0]
    
y_test_hat = np.argmax(y_test_hat[:,0,:], axis = -1)
y_test_hat

array([0, 0, 0, ..., 0, 1, 0])

In [24]:
## Squeeze and argmax Target into 1d
if len(y_val.shape) == 3:
    y_val = np.argmax(y_val[:,0,:], axis=-1)
    
if len(y_train.shape) == 3:
    y_train = np.argmax(y_train[:,0,:], axis=-1)


# Training Metrics

In [25]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
cm = confusion_matrix(y_train, y_train_hat)
rp = classification_report(y_train, y_train_hat)
score = accuracy_score(y_train, y_train_hat)

print(cm)
print(rp)
print(score)

[[82456  7275]
 [ 7897 86832]]
              precision    recall  f1-score   support

           0       0.91      0.92      0.92     89731
           1       0.92      0.92      0.92     94729

   micro avg       0.92      0.92      0.92    184460
   macro avg       0.92      0.92      0.92    184460
weighted avg       0.92      0.92      0.92    184460

0.9177491054971267


# Validation Metrics

In [26]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
cm = confusion_matrix(y_val, y_val_hat)
rp = classification_report(y_val, y_val_hat)
score = accuracy_score(y_val, y_val_hat)
f1 = f1_score(y_val, y_val_hat)

print(cm)
print(rp)
print('accuracy : ', score)
print('f1_score : ', f1)

[[9734 3288]
 [3104 9734]]
              precision    recall  f1-score   support

           0       0.76      0.75      0.75     13022
           1       0.75      0.76      0.75     12838

   micro avg       0.75      0.75      0.75     25860
   macro avg       0.75      0.75      0.75     25860
weighted avg       0.75      0.75      0.75     25860

accuracy :  0.7528228924980666
f1_score :  0.7528228924980667


# Testing metrics

In [27]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_test_hat)
rp = classification_report(y_test, y_test_hat)
score = accuracy_score(y_test, y_test_hat)

print(cm)
print(rp)
print(score)

[[3404 1006]
 [1137 3034]]
              precision    recall  f1-score   support

           0       0.75      0.77      0.76      4410
           1       0.75      0.73      0.74      4171

   micro avg       0.75      0.75      0.75      8581
   macro avg       0.75      0.75      0.75      8581
weighted avg       0.75      0.75      0.75      8581

0.7502622072019578
